In [ ]:
FILE="echantillon_1000_hs_2024_TOC.parquet"

import pandas as pd
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_text_splitters import CharacterTextSplitter
from tqdm import tqdm

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=3000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)



model_kwargs = {'device': 'cuda'}
embedder = HuggingFaceEmbeddings(model_name="BAAI/bge-m3", model_kwargs=model_kwargs,show_progress=False)

df=pd.read_parquet(FILE)
df=df.rename(columns={"accorddocx":"texte"})
df=df.set_index("numdossier_new")

vector_store = Chroma(embedding_function=embedder, persist_directory="./chroma_db")
for index, row in tqdm(df.iterrows(), total=len(df)):
    text=df.loc[index].texte
    texts = text_splitter.create_documents([text])
    i=0
    for t in texts:
        t.metadata["id"]=f"{index}_{i}"
        t.metadata["index"]=f"{index}"
        vector_store.add_documents([t])
        i+=1